# WeRateDogs Twitter Archive Wrangling Project

## Background
WeRateDogs is a social media page that rates people’s dogs with humorous comments. It was founded by Matt Nelson in 2015 and has since become an authority in its niche. Aside from rating people’s dogs, the page also helps animal rights organisations such as ASPCA (American Society for the Prevention of Cruelty to Animals) raise funds and run GoFundMe campaigns for dogs. 


## Gathering Data
The main data required for this project was WeRateDogs’ Twitter archive, which WeRateDog itself provided to Udacity.
Two additional datasets were generated based on the Twitter archive.

**Image prediction**: This data was generated by running every image in the WeRateDogs Twitter archive through a neural network that can classify breeds of dogs.

The variables in this data are:

- tweet_id is the last part of the tweet URL after "status/"
- jpg_url is the image URL
- img_num is the image number corresponding to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).
- p1 is the algorithm's #1 prediction for the image in the tweet
- p1_conf is how confident the algorithm is in its #1 prediction
- p1_dog is whether or not the #1 prediction is a breed of dog
- p2 is the algorithm's second most likely prediction
- p2_conf is how confident the algorithm is in its #2 prediction 
- p2_dog is whether or not the #2 prediction is a breed of dog 
- p3 is the algorithm's third most likely prediction
- p3_conf is how confident the algorithm is in its #3 prediction 
- p3_dog is whether or not the #3 prediction is a breed of dog

**Tweet_df**: This data was generated from querying Twitter API for WeRateDogs’ Twitter archive 

The variables in this data are:

- tweet_id
- retweet_count is the number of times a tweet was retweeted
- favourite_count is the number of times a tweet was favourited. 




## Assessing Data
The author employed both visual and programmatic assessments to review the three pieces of data for quality and tidiness issues. A methodical approach was taken to visually and programmatically assess each of the data on Microsoft Excel; reviewing each table a column at a time.  

With visual assessment, the following issues were discovered:

**Data quality**;
- source variable appear as ``<a>`` tag instead of device type;
- expanded_urls contain multiple urls in one row;
- in_reply_to_status_id and in_reply_to_user_id contain rows that are replies and not original tweets;
- retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp contains rows that are retweets and not original tweets 
- The breed of some dog images were not predicted. That is p1_dog, p2_dog, and p3_dog are all false

**Tidiness issues**;
- Some tweets had more than one dog stage; and
- One variable (dog stage) in three columns (doggo, floofer, pupper, puppo).
- Two variables in nine columns (dog breed and prediction confidence)
- ``image_prediction`` should be part of ``twitter_archive`` table
- ``tweet_df`` should be part of ``twitter_archive`` table

The author used `describe()`, `sample()`, and `info()` functions to programmatically assess the three data. Here are what were discovered:

**Data quality**;
- Erroneous datatypes for tweet_id, rating_numerator, and timestamp;
- a, an, very, just, by, my, o, this as dog names;
- Digits after the decimal point were omitted for rating_numerator; and
- Missing records in ``image_prediction``: 2075 instead of 2356.



## Cleaning
Before the author began cleaning the data, the author made copies of the original datasets, using ``copy()`` function:
```python
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_df_clean = tweet_df.copy()
```

Cleaning was performed one table at a time, starting from ``twitter_archive ``to ``tweet_df``

### ``twitter_archive``: data quality and tidiness resolutions

#### First, the tidiness issue of one variable (`dog_stage`) in four columns and the data quality issue of tweets having multiple dog stages were resolved. 

1. A new table (``dog_stage_df``) was created by applying ``pd.melt()`` on columns tweet_id, doggo, floofer, pupper, puppo
    ```python
    dog_stage_df = pd.melt(twitter_archive_clean, id_vars='tweet_id', value_vars=['doggo', 'floofer', 'pupper', 'puppo'], value_name='dog_stage')
    ```

2. The variable column and duplicates rows resulting from melting those columns were dropped using ``drop()`` and ``drop_duplicates()``, and None values in the dog_stage column were then replaced with ``np.NAN``
    
    ```python
    dog_stage_df.drop('variable', axis=1, inplace=True)
    dog_stage_df.drop_duplicates(inplace=True)
    dog_stage_df['dog_stage'] = dog_stage_df['dog_stage'].replace('None', np.NAN)
    ```

3. The author combined the stages for tweets with more than one dog stage and dropped the duplicate values in the ``dog_stage_df`` table

    ```python
    with_dog_stages = dog_stage_df[dog_stage_df.dog_stage.notna()]
    combined_stages = with_dog_stages.groupby('tweet_id').transform(lambda x: ', '.join(x))
    dog_stage_df.loc[combined_stages.index, 'dog_stage'] = combined_stages.values
    dog_stage_df.sort_values(by='dog_stage', inplace=True)
    dog_stage_df.drop_duplicates(subset='tweet_id', inplace=True)
    ```

4. ``dog_stage_df`` was merged to the twitter_archive using ``pd.merge()`` with left join

    ```python
    twitter_archive_clean = pd.merge(twitter_archive_clean, dog_stage_df, how='left', on='tweet_id')
    twitter_archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)
    ```


#### Second, the author addressed the data quality issue of tweets that were replies and retweets  

1. Rows that were replies were removed using ``isnull()`` and columns in_reply_to_status_id and in_reply_to_user_id were dropped
    
    ```python
    twitter_archive_clean = twitterarchive_clean[twitter_archive_clean['in_reply_to_user_id'].isnull()]
    twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace=True)
    ```

2. Rows that were retweets were removed using ``isnull()`` and columns retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp were dropped

    ```python
    twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['retweeted_status_user_id'].isnull()]
    twitter_archive_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)
    ```

#### Third, the author dealt with the data quality issue of digits after the decimal point being omitted for the variable ``rating_numerator``

1. The rating numerator for each tweet was extracted from the tweets using extact() and regex

    ```python
    twitter_archive_clean['rating_numerator'] = twitter_archive_clean['text'].str.extract('(\d+\.*\d*\/\d+)', expand=False).str.split('/').str[0]
    ```



#### Finally, the last two data quality issues (source is coded as ``<a>`` tag instead of device type and expanded urls contain multiple urls in one row) in the ``twitter_archive`` table were resolved

1. The device types were extracted from the source url using ``extract()`` and regex
    
    ```python
    twitter_archive_clean['source'] = twitter_archive_clean['source'].str.extract('^<a.+>(.+)</a>$')
    ```

2. The first urls in expanded_url's rows were retained and the others removed using a combination of ``split()`` and list slicing

    ```python
    twitter_archive_clean['expanded_urls'] = twitter_archive_clean['expanded_urls'].str.split(',').str[0]
    ```

### ``image_prediction``: data quality and tidiness resolutions

#### First, the author addressed the tidiness issue: two variables in nine columns (dog breed and prediction confidence)

The issue was resolved using ``np.select()``, which takes a condition list and a choice list.

1. The author created a condition list for dog breed and a choice list for dog breed and prediction confidence

    ```python
    condlist = [(image_prediction_clean['p1_dog'] == True), (image_prediction_clean['p2_dog'] == True), (image_prediction_clean['p3_dog'] == True)]
    choicelist = [image_prediction_clean['p1'], image_prediction_clean['p2'], image_prediction_clean['p3']]
    choicelist_2 = [image_prediction_clean['p1_conf'], image_prediction_clean['p2_conf'], image_prediction_clean['p3_conf']]
    ```

2. Then ``np.select()`` was used to create dog_breed and picture columns variables

    ```python
    image_prediction_clean['breed'] = np.select(condlist, choicelist, default='Unknown')
    image_prediction_clean['pix_conf'] = np.select(condlist, choicelist_2, default=0)
    ```

3. The columns in ``image_prediction`` table were dropped

    ```python
    image_prediction_clean.drop(['img_num', 'p1', 'p1_dog', 'p1_conf', 'p2', 'p2_dog', 'p2_conf', 'p3', 'p3_dog', 'p3_conf'], axis=1, inplace=True)
    ```

#### Second, ``image_prediction`` table was merged with the ``twitter_archive table``

1. The author used ``pd.merge()`` to left merge ``twitter_archive`` to ``image_prediction`` using tweet_id

    ```python
    twitter_archive_clean = pd.merge(twitter_archive_clean, image_prediction_clean, how='left', on='tweet_id')
    ```

#### The author went back to the ``twitter_archive`` table after merging it with ``image_prediction`` table to fix the erroneous datatype issues

1. The author set the right datatype for tweet_id, rating_numerator, and dog_stage using ``astype()``

    ```python
    twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
    twitter_archive_clean['rating_numerator'] = twitter_archive_clean['rating_numerator'].astype(float)
    twitter_archive_clean[['dog_stage', 'breed']] = twitter_archive_clean[['dog_stage', 'breed']].astype('category')
    ```

2. timestamp was converted to datetype using ``pd.to_datetype()``

    ```python
    twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])
    ```

### ``tweet_df``: data quality and tidiness resolutions

1. The only issue the author had to fix when it comes to tweet_df is left merging it to twitter_archive

    ```python
    twitter_archive_clean = pd.merge(twitter_archive_clean, tweet_df_clean, how='left', on='tweet_id')
    ```


With that done, all data quality and tidiness issues with the dataset were resolved.

## Storing Data

#### The author saved the master dataset to a sqlite3 database

1. Imported sqlite3, defined table name, and set the column names

    ```python
    import sqlite3
    table_name = 'master'
    col_names = ['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
        'rating_numerator', 'rating_denominator', 'name', 'dog_stage',
        'jpg_url', 'breed', 'pix_conf', 'retweet_count', 'favorite_count']
    ```

2. The sqlite3 connection and query were defined and to_sql() was used to store the dataset in a sqlite3 database

    ```python
    conn = sqlite3.connect('twitter_archive_master.sqlite')
    query = f'Create table if not Exists {table_name} (col_names)'
    conn.execute(query)
    twitter_archive_clean.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    ```